In [2]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoModel
!pip install sklearn
from sklearn.model_selection import train_test_split
import transformers
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold , KFold

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 40.1 MB/s eta 0:00:0000:0100:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=9ea6719ad20f270d5de53f09c80a5712914ef6fe093bf122f72687cd237bd7cd
  Stored in directory: /opt/ml/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [3]:
!gpustat

e9164cf4a246             Tue Nov  1 06:35:05 2022  450.80.02
[0] Tesla V100-PCIE-32GB | 38°C,   0 % |  2319 / 32510 MB |


In [4]:
train_data = pd.read_csv('../data/train.csv')
val_data = pd.read_csv('../data/dev.csv')


In [7]:
print(len(train_data))
print(len(val_data))
print(len(total_data))

9324
550


NameError: name 'total_data' is not defined

In [8]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

In [5]:
    def prepare_input(self,text):
        inputs = self.tokenizer.encode_plus(text, return_tensors=None, add_special_tokens=True, padding = 'max_length',truncation=True)
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long) # 텐서로 변환
        return inputs

In [41]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained('klue/roberta-small', max_length=160) 
        self.texts = (data['sentence_1'] + '[SEP]' + data['sentence_2']).values
        if 'label' in data.columns:
            self.targets = data['label'].values
        else:
            self.targets = []

    def prepare_input(self,text):
        inputs = self.tokenizer.encode(text, return_tensors=None,
        add_special_tokens=True, padding = 'max_length',truncation=True)
        # for k, v in inputs.items():
        #     inputs[k] = torch.tensor(v, dtype=torch.long) # 텐서로 변환
        return inputs

    # 학습 및 추론 과정에서 데이터를 1개씩 꺼내오는 곳
    def __getitem__(self, idx):
        inputs = self.prepare_input(self.texts[idx])
        # 정답이 있다면 else문을, 없다면 if문을 수행합니다
        if len(self.targets) == 0:
            return inputs
        else:
            label = torch.tensor(self.targets[idx],dtype=torch.float)
            return inputs, label

    # 입력하는 개수만큼 데이터를 사용합니다
    def __len__(self):
        return len(self.texts)

In [13]:
total_data = pd.concat([train_data,val_data],axis = 0)
total_data['simple_label'] = total_data['label'].apply(lambda x : int(x))
total_data

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,2
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,4
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,2
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,3
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,0
...,...,...,...,...,...,...,...
545,boostcamp-sts-v1-dev-545,nsmc-sampled,"사회적 이슈를 다루고 있는 가슴 찡한 드라마네요,,,",정말 가슴을 따뜻하게 한 좋은 드라마...,2.0,0.0,2
546,boostcamp-sts-v1-dev-546,slack-rtt,(비타민 먹는 장면),(비타민을 먹는 장면),4.8,1.0,4
547,boostcamp-sts-v1-dev-547,nsmc-sampled,내용이 뭔 내용인지도 모르겠음,무슨의미로 만들었는지 모르겠음..,2.4,0.0,2
548,boostcamp-sts-v1-dev-548,slack-rtt,"(예: 주말에는 개인캘린더만, 업무시간에는 업무 캘린더만 보기)","(예: 주말에는 개인캘린더만 보고, 업무시간에는 업무캘린더만 보기)",5.0,1.0,5


In [15]:
total_data = total_data.reset_index(drop=True)

In [16]:
total_data

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,2
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,4
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,2
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,3
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,0
...,...,...,...,...,...,...,...
9869,boostcamp-sts-v1-dev-545,nsmc-sampled,"사회적 이슈를 다루고 있는 가슴 찡한 드라마네요,,,",정말 가슴을 따뜻하게 한 좋은 드라마...,2.0,0.0,2
9870,boostcamp-sts-v1-dev-546,slack-rtt,(비타민 먹는 장면),(비타민을 먹는 장면),4.8,1.0,4
9871,boostcamp-sts-v1-dev-547,nsmc-sampled,내용이 뭔 내용인지도 모르겠음,무슨의미로 만들었는지 모르겠음..,2.4,0.0,2
9872,boostcamp-sts-v1-dev-548,slack-rtt,"(예: 주말에는 개인캘린더만, 업무시간에는 업무 캘린더만 보기)","(예: 주말에는 개인캘린더만 보고, 업무시간에는 업무캘린더만 보기)",5.0,1.0,5


In [17]:

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state=22)
for n_fold, (_,v_idx) in enumerate(skf.split(total_data,total_data['simple_label'])):
    total_data.loc[v_idx,'fold'] = n_fold
total_data

,id,source,sentence_1,sentence_2,label,binary-label,simple_label,fold
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,2,2.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,4,4.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,2,3.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,3,4.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,0,4.0
...,...,...,...,...,...,...,...,...
9869,boostcamp-sts-v1-dev-545,nsmc-sampled,"사회적 이슈를 다루고 있는 가슴 찡한 드라마네요,,,",정말 가슴을 따뜻하게 한 좋은 드라마...,2.0,0.0,2,3.0
9870,boostcamp-sts-v1-dev-546,slack-rtt,(비타민 먹는 장면),(비타민을 먹는 장면),4.8,1.0,4,4.0
9871,boostcamp-sts-v1-dev-547,nsmc-sampled,내용이 뭔 내용인지도 모르겠음,무슨의미로 만들었는지 모르겠음..,2.4,0.0,2,0.0
9872,boostcamp-sts-v1-dev-548,slack-rtt,"(예: 주말에는 개인캘린더만, 업무시간에는 업무 캘린더만 보기)","(예: 주말에는 개인캘린더만 보고, 업무시간에는 업무캘린더만 보기)",5.0,1.0,5,1.0


In [18]:
train_data = total_data[total_data['fold'] != 0]
val_data = total_data[total_data['fold'] == 0]

In [19]:
train_data['simple_label'].value_counts()

0    3039
3    1460
1    1182
4    1130
2     997
5      91
Name: simple_label, dtype: int64

In [20]:
val_data['simple_label'].value_counts()

0    760
3    365
1    296
4    282
2    250
5     22
Name: simple_label, dtype: int64

In [21]:
train_data['source'].value_counts()

petition-sampled    2015
nsmc-sampled        1910
slack-sampled       1687
slack-rtt            909
petition-rtt         713
nsmc-rtt             665
Name: source, dtype: int64

In [36]:
train_data[['source','simple_label']].value_counts()

source            simple_label
slack-sampled     0               1053
petition-sampled  0                960
nsmc-sampled      0                916
                  1                398
slack-rtt         3                376
petition-sampled  1                364
petition-rtt      4                334
slack-rtt         4                316
nsmc-sampled      2                311
slack-sampled     1                305
nsmc-rtt          3                300
petition-sampled  3                248
                  2                240
petition-rtt      3                216
nsmc-sampled      3                213
nsmc-rtt          4                202
slack-sampled     2                167
petition-sampled  4                164
slack-rtt         2                117
slack-sampled     3                107
nsmc-rtt          2                 91
petition-rtt      2                 71
nsmc-sampled      4                 70
slack-rtt         1                 52
slack-sampled     4              

In [38]:
train_data.duplicated(['sentence_1', 'sentence_2']).sum()

0

In [2]:
!pip install tensorboardX
from pororo import Pororo

  Using cached tensorboardX-2.5.1-py2.py3-none-any.whl (125 kB)


In [3]:
from pororo import Pororo
mt = Pororo(task='translation', lang = 'multi')

def back_trans_pororo(text,lang='en'):
    text_to_lang = mt(text, src='ko', tgt=lang)
    new_text = mt(text_to_lang,src=lang,tgt='ko')
    return new_text

In [4]:
mt = Pororo(task='translation', lang = 'multi')

In [5]:
mt('hello', src='en', tgt='ko')

ValueError: Please build (or rebuild) Cython components with `python setup.py build_ext --inplace`.

In [ ]:
Pororo.available_tasks()

"Available tasks are ['mrc', 'rc', 'qa', 'question_answering', 'machine_reading_comprehension', 'reading_comprehension', 'sentiment', 'sentiment_analysis', 'nli', 'natural_language_inference', 'inference', 'fill', 'fill_in_blank', 'fib', 'para', 'pi', 'cse', 'contextual_subword_embedding', 'similarity', 'sts', 'semantic_textual_similarity', 'sentence_similarity', 'sentvec', 'sentence_embedding', 'sentence_vector', 'se', 'inflection', 'morphological_inflection', 'g2p', 'grapheme_to_phoneme', 'grapheme_to_phoneme_conversion', 'w2v', 'wordvec', 'word2vec', 'word_vector', 'word_embedding', 'tokenize', 'tokenise', 'tokenization', 'tokenisation', 'tok', 'segmentation', 'seg', 'mt', 'machine_translation', 'translation', 'pos', 'tag', 'pos_tagging', 'tagging', 'const', 'constituency', 'constituency_parsing', 'cp', 'pg', 'collocation', 'collocate', 'col', 'word_translation', 'wt', 'summarization', 'summarisation', 'text_summarization', 'text_summarisation', 'summary', 'gec', 'review', 'review_s